<a href="https://colab.research.google.com/github/segedakate2005/-/blob/main/%D0%BB%D0%B0%D0%B1_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy numpy==1.26.4
!pip install surprise

In [ ]:
import numpy as np
from surprise import Dataset
import os
import math
import random

In [ ]:
data = Dataset.load_builtin('ml-100k')

print("Перші 5 рядків датасету:")
for row in data.raw_ratings[:5]:
    print(row)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
Перші 5 рядків датасету:
('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')


In [ ]:
raw_ratings = data.raw_ratings

# --- 2. Load movie titles manually ---
# Find the path to the dataset files downloaded by surprise
# This assumes the default download location (~/.surprise_data/ml-100k/)
# You might need to adjust this path if it's different on your system.
try:
    data_folder = os.path.expanduser('~/.surprise_data/ml-100k/ml-100k/')
    item_file_path = os.path.join(data_folder, 'u.item')

    # Check if the file exists, if not, maybe try fetching it (though load_builtin should handle it)
    if not os.path.exists(item_file_path):
         print(f"Warning: u.item not found at {item_file_path}. Movie titles will not be available.")
         item_id_to_title = {}
    else:
         item_id_to_title = {}
         with open(item_file_path, 'r', encoding='ISO-8859-1') as f:
             for line in f:
                 parts = line.strip().split('|')
                 item_id = int(parts[0])
                 title = parts[1]
                 item_id_to_title[item_id] = title
         print(f"Loaded {len(item_id_to_title)} movie titles.")

except FileNotFoundError:
     print("Warning: Could not find the ml-100k data folder. Movie titles will not be available.")
     print("Please ensure the dataset was downloaded correctly by surprise.")
     item_id_to_title = {}


# --- 3. Determine matrix dimensions ---
max_user_id = 0
max_item_id = 0
for uid, iid, _, _ in raw_ratings:
    user_id = int(uid)
    item_id = int(iid)
    if user_id > max_user_id:
        max_user_id = user_id
    if item_id > max_item_id:
        max_item_id = item_id

num_users = max_user_id
num_items = max_item_id
print(f"Found {num_users} users and {num_items} items.")

# --- 4. Build the rating matrix ---
# Initialize matrix with zeros
ratings = np.zeros((num_users, num_items))

# Populate the matrix
# Note: raw_ratings gives user/item IDs as strings, convert to int
# We use 0-based indexing for the matrix (user_id - 1, item_id - 1)
for uid, iid, rating, _ in raw_ratings:
    user_index = int(uid) - 1
    item_index = int(iid) - 1
    # Ensure indices are within bounds (although they should be given how we found max IDs)
    if 0 <= user_index < num_users and 0 <= item_index < num_items:
         ratings[user_index, item_index] = float(rating)
    else:
         print(f"Warning: Index out of bounds for user {uid} or item {iid}")

Loaded 1682 movie titles.
Found 943 users and 1682 items.


In [ ]:
# Гіперпараметри
num_factors = 10
steps = 30
alpha = 0.01
lambda_reg = 0.1

# Ініціалізація матриць
# Using lists of lists as in the example
def init_matrix(rows, cols):
    # Small random values are often better for initialization
    # return [[random.uniform(0, 0.1) for _ in range(cols)] for _ in range(rows)]
    # Or use numpy for potentially better performance if allowed (example used lists)
     return np.random.rand(rows, cols) * 0.1 # Using numpy initialization

U = init_matrix(num_users, num_factors)
V = init_matrix(num_items, num_factors) # Note: shape is (num_items, num_factors)

# Навчання (ALS-подібний підхід) - using the example's SGD-like update
print("Starting training...")
for step in range(steps):
    print(f"Step {step+1}/{steps}")
    for i in range(num_users):
        for j in range(num_items):
            r_ij = ratings[i][j]
            if r_ij == 0: # Skip if no rating exists
                continue

            # Predict
            pred = np.dot(U[i, :], V[j, :]) # Using numpy dot product
            # pred = sum(U[i][k] * V[j][k] for k in range(num_factors)) # Original list version

            # Calculate error
            error = r_ij - pred

            # Update factors using Gradient Descent logic
            for k in range(num_factors):
                 u_ik = U[i, k]
                 v_jk = V[j, k]
                 U[i, k] += alpha * (error * v_jk - lambda_reg * u_ik)
                 V[j, k] += alpha * (error * u_ik - lambda_reg * v_jk) # Corrected index to V[j, k]
print("Training finished.")

# Прогноз
def predict_rating(user_index, item_index):
    # Ensure indices are within bounds before predicting
    if 0 <= user_index < num_users and 0 <= item_index < num_items:
         # return sum(U[user_index][k] * V[item_index][k] for k in range(num_factors)) # Original list version
         return np.dot(U[user_index, :], V[item_index, :]) # Using numpy dot product
    else:
         print(f"Warning: Index out of bounds for prediction ({user_index}, {item_index})")
         return 0 # Return a default value like 0 or average rating

# --- 5. Adapt the recommendation function ---
def recommend(user_id_input, top_n=5):
    user_index = user_id_input - 1 # Convert 1-based user_id to 0-based index

    if not (0 <= user_index < num_users):
        print(f"Error: User ID {user_id_input} is out of range.")
        return

    predictions = []
    for j in range(num_items): # j is the 0-based item index
        # Check if the user has NOT rated this item
        if ratings[user_index, j] == 0:
            pred = predict_rating(user_index, j)
            predictions.append((j, pred)) # Store 0-based index and prediction

    # Sort by predicted rating in descending order
    predictions.sort(key=lambda x: x[1], reverse=True)

    print(f"\nРекомендації для користувача {user_id_input}:")
    for idx in range(min(top_n, len(predictions))):
        item_index, score = predictions[idx] # item_index is 0-based
        item_id = item_index + 1 # Convert back to 1-based item_id for lookup

        # Get title from the dictionary
        title = item_id_to_title.get(item_id, f"Unknown Title (ID: {item_id})")

        print(f"→ {title} (ID: {item_id}) — прогнозована оцінка: {round(score, 2)}")

# Приклад - Recommend for user 1
recommend(1)

# Example - Recommend for user 196 (who rated item 242 in the first few lines of data)
recommend(196, top_n=10)

Starting training...
Step 1/30
Step 2/30
Step 3/30
Step 4/30
Step 5/30
Step 6/30
Step 7/30
Step 8/30
Step 9/30
Step 10/30
Step 11/30
Step 12/30
Step 13/30
Step 14/30
Step 15/30
Step 16/30
Step 17/30
Step 18/30
Step 19/30
Step 20/30
Step 21/30
Step 22/30
Step 23/30
Step 24/30
Step 25/30
Step 26/30
Step 27/30
Step 28/30
Step 29/30
Step 30/30
Training finished.

Рекомендації для користувача 1:
→ Pather Panchali (1955) (ID: 1449) — прогнозована оцінка: 4.79
→ Close Shave, A (1995) (ID: 408) — прогнозована оцінка: 4.67
→ Faust (1994) (ID: 1367) — прогнозована оцінка: 4.64
→ Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963) (ID: 474) — прогнозована оцінка: 4.53
→ Two or Three Things I Know About Her (1966) (ID: 851) — прогнозована оцінка: 4.49

Рекомендації для користувача 196:
→ Pather Panchali (1955) (ID: 1449) — прогнозована оцінка: 4.9
→ Faust (1994) (ID: 1367) — прогнозована оцінка: 4.64
→ Wrong Trousers, The (1993) (ID: 169) — прогнозована оцінка: 4.6
→ Schind

In [ ]:
# Градієнтний спуск
for step in range(steps):
    for i in range(num_users):
        for j in range(num_items):
            r_ij = ratings[i][j]
            if r_ij == 0:
                continue
            # передбачення та помилка
            pred = sum(U[i][k] * V[j][k] for k in range(num_factors))
            error = r_ij - pred
            # оновлення U і V
            for k in range(num_factors):
                u_ik = U[i][k]
                v_jk = V[j][k]
                U[i][k] += alpha * (error * v_jk - lambda_reg * u_ik)
                V[j][k] += alpha * (error * u_ik - lambda_reg * v_jk)

# Прогноз
def predict_rating(user_index, item_index):
    return sum(U[user_index][k] * V[item_index][k] for k in range(num_factors))

# Рекомендації
def recommend(user_index, top_n=5):
    preds = []
    rated_items = [int(iid) - 1 for uid, iid, _, _ in raw_ratings if int(uid) - 1 == user_index]
    for j in range(num_items):
        if j not in rated_items:
            pred = predict_rating(user_index, j)
            preds.append((j, pred))
    preds.sort(key=lambda x: x[1], reverse=True)
    print(f"Рекомендації для користувача {user_index + 1}:")
    count = 0
    for item_index, score in preds:
        if item_index + 1 in item_id_to_title:
            print(f"Фільм: {item_id_to_title[item_index + 1]}, Оцінка: {round(score, 2)}")
            count += 1
            if count == top_n:
                break
        elif count < top_n:
            print(f"Товар ID: {item_index + 1}, Оцінка: {round(score, 2)} (назва недоступна)")
            count += 1

# Тест для перших 5 користувачів
for i in range(min(5, num_users)):
    recommend(i)
    print("-" * 30)

Рекомендації для користувача 1:
Фільм: Santa with Muscles (1996), Оцінка: 4.92
Фільм: Pather Panchali (1955), Оцінка: 4.87
Фільм: Faust (1994), Оцінка: 4.84
Фільм: Close Shave, A (1995), Оцінка: 4.73
Фільм: Saint of Fort Washington, The (1993), Оцінка: 4.68
------------------------------
Рекомендації для користувача 2:
Фільм: Pather Panchali (1955), Оцінка: 4.93
Фільм: Santa with Muscles (1996), Оцінка: 4.89
Фільм: Saint of Fort Washington, The (1993), Оцінка: 4.7
Фільм: Schindler's List (1993), Оцінка: 4.69
Фільм: Some Mother's Son (1996), Оцінка: 4.66
------------------------------
Рекомендації для користувача 3:
Фільм: Santa with Muscles (1996), Оцінка: 4.34
Фільм: Star Wars (1977), Оцінка: 4.26
Фільм: Blade Runner (1982), Оцінка: 4.24
Фільм: Pather Panchali (1955), Оцінка: 4.21
Фільм: Godfather, The (1972), Оцінка: 4.16
------------------------------
Рекомендації для користувача 4:
Фільм: Santa with Muscles (1996), Оцінка: 6.2
Фільм: Pather Panchali (1955), Оцінка: 6.08
Фільм: Faus

In [ ]:
# Функція обчислення відстані між двома користувачами на основі їх оцінок
def distance(u1, u2):
    total = 0
    count = 0
    for i in range(len(u1)):
        if u1[i] != 0 and u2[i] != 0:
            total += (u1[i] - u2[i]) ** 2
            count += 1
    if count == 0:
        return float('inf')
    return math.sqrt(total / count)

# Ініціалізуємо центроїди випадковими користувачами
random_user_indices = random.sample(range(num_users), k)
centroids = [ratings[i].copy() for i in random_user_indices]

# Кластеризація
clusters = [[] for _ in range(k)]

for _ in range(20):  # Збільшено кількість ітерацій для кращої збіжності
    clusters = [[] for _ in range(k)]
    for user_index, user_ratings in enumerate(ratings):
        min_dist = float('inf')
        cluster_id = -1
        for c_index, centroid in enumerate(centroids):
            dist = distance(user_ratings, centroid)
            if dist < min_dist:
                min_dist = dist
                cluster_id = c_index
        if cluster_id != -1:
            clusters[cluster_id].append(user_index)

    # Перерахунок центроїдів
    new_centroids = [[0.0] * num_items for _ in range(k)]
    cluster_counts = [0] * k
    for c_index, user_indices in enumerate(clusters):
        for u_index in user_indices:
            for item_index in range(num_items):
                if ratings[u_index][item_index] != 0:
                    new_centroids[c_index][item_index] += ratings[u_index][item_index]
        cluster_counts[c_index] = len(user_indices)
        if cluster_counts[c_index] > 0:
            new_centroids[c_index] = [val / cluster_counts[c_index] if cluster_counts[c_index] > 0 else 0 for val in new_centroids[c_index]]

    # Перевірка на збіжність (якщо центроїди майже не змінилися)
    centroid_diff = 0
    for c in range(k):
        centroid_diff += distance(centroids[c], new_centroids[c])
    centroids = new_centroids
    if centroid_diff < 0.001:  # Поріг збіжності
        print("Кластеризація зійшлася.")
        break
else:
    print("Кластеризація завершилася після заданої кількості ітерацій.")

# Рекомендації на основі кластерів
def recommend(user_index, top_n=5):
    user_ratings = ratings[user_index]
    # Знайдемо найближчий центроїд
    min_dist = float('inf')
    best_cluster_index = -1
    for c_index, centroid in enumerate(centroids):
        dist = distance(user_ratings, centroid)
        if dist < min_dist:
            min_dist = dist
            best_cluster_index = c_index

    if best_cluster_index == -1:
        print(f"Не вдалося знайти кластер для користувача {user_index + 1}.")
        return

    centroid_ratings = centroids[best_cluster_index]
    preds = []
    rated_items = [i for i, rating in enumerate(user_ratings) if rating != 0]

    for item_index in range(num_items):
        if item_index not in rated_items:
            preds.append((item_index, centroid_ratings[item_index]))

    preds.sort(key=lambda x: x[1], reverse=True)
    print(f"Рекомендації для користувача {user_index + 1} (на основі кластера {best_cluster_index + 1}):")
    count = 0
    for item_index, score in preds:
        if item_index + 1 in item_id_to_title:
            print(f"Фільм: {item_id_to_title[item_index + 1]}, Середня оцінка кластера: {round(score, 2)}")
            count += 1
            if count == top_n:
                break
        elif count < top_n:
            print(f"Товар ID: {item_index + 1}, Середня оцінка кластера: {round(score, 2)} (назва недоступна)")
            count += 1

# Тестування для перших 5 користувачів
for i in range(min(5, num_users)):
    recommend(i)
    print("-" * 30)

Кластеризація завершилася після заданої кількості ітерацій.
Рекомендації для користувача 1 (на основі кластера 7):
Фільм: Schindler's List (1993), Середня оцінка кластера: 2.17
Фільм: E.T. the Extra-Terrestrial (1982), Середня оцінка кластера: 2.08
Фільм: One Flew Over the Cuckoo's Nest (1975), Середня оцінка кластера: 1.9
Фільм: Casablanca (1942), Середня оцінка кластера: 1.76
Фільм: It's a Wonderful Life (1946), Середня оцінка кластера: 1.73
------------------------------
Рекомендації для користувача 2 (на основі кластера 1):
Фільм: Return of the Jedi (1983), Середня оцінка кластера: 2.25
Фільм: Independence Day (ID4) (1996), Середня оцінка кластера: 2.05
Фільм: Rock, The (1996), Середня оцінка кластера: 1.93
Фільм: Mr. Holland's Opus (1995), Середня оцінка кластера: 1.85
Фільм: Twelve Monkeys (1995), Середня оцінка кластера: 1.81
------------------------------
Рекомендації для користувача 3 (на основі кластера 5):
Фільм: Titanic (1997), Середня оцінка кластера: 2.88
Фільм: English P